In [8]:
import kaggle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

1. Download the data programatically with python.

In [5]:
kaggle.api.authenticate()
kaggle.api.dataset_download_files('PromptCloudHQ/imdb-data', path='imdb-data', unzip=True)

2. Fit the training-data into a model, present it visually and reflect on wether it is a good fit or not.

In [65]:
data = pd.read_csv('imdb-data/IMDB-Movie-Data.csv')
data.head()

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0


In [66]:
revenue = data.pop('Revenue (Millions)').values
revenue

array([3.3313e+02, 1.2646e+02, 1.3812e+02, 2.7032e+02, 3.2502e+02,
       4.5130e+01, 1.5106e+02,        nan, 8.0100e+00, 1.0001e+02,
       2.3402e+02, 1.6927e+02, 5.3217e+02, 2.4875e+02, 2.8700e+00,
       3.6831e+02, 6.7120e+01, 1.6216e+02, 5.1690e+01, 1.0050e+02,
       7.2200e+00, 4.7700e+01,        nan, 1.5369e+02, 1.0314e+02,
              nan, 6.5000e+00, 1.0000e-02, 1.1308e+02, 5.4650e+01,
       6.0310e+01, 1.0640e+01, 1.5533e+02, 3.6302e+02, 2.6840e+01,
       4.0808e+02, 1.8799e+02, 2.3260e+02, 9.3380e+01,        nan,
       9.7660e+01, 2.7850e+01,        nan, 1.2790e+01, 4.2100e+00,
       2.4106e+02, 3.4400e+00,        nan, 1.5880e+02,        nan,
       9.3663e+02, 3.0350e+01, 3.2460e+01, 4.3000e+01, 5.3332e+02,
       7.0800e+00, 8.9210e+01, 5.6230e+01, 3.1800e+00, 1.2507e+02,
       3.3025e+02,        nan, 7.5310e+01, 1.6615e+02, 5.3080e+01,
       1.2825e+02, 3.1860e+01, 1.5363e+02, 1.0000e-02, 6.1280e+01,
              nan, 4.0070e+01, 3.7300e+00, 3.0980e+01, 3.4126e

In [63]:
from sklearn.preprocessing import OneHotEncoder

def encode(frame, column):
    column_train = frame[[column]].copy()
    ohe = OneHotEncoder(sparse=False)
    column_train_transformed = ohe.fit_transform(column_train)
    return column_train_transformed, ohe

In [86]:
encoded_titles, title_ohe = encode(data, "Title")
encoded_directors, director_ohe = encode(data, "Director")
encoded_genres, genre_ohe = encode(data, "Genre")
encoded_descriptions, description_ohe = encode(data, "Description")
encoded_actors, actors_ohe= encode(data, "Actors")

It’s good to verify that our estimator is working properly. Let’s look at the first row of encoded data.

In [89]:
feature_names = title_ohe.get_feature_names()
first = feature_names[encoded_titles[0] == 1]
print(first)
print(data['Title'][0])

['x0_Guardians of the Galaxy']
Guardians of the Galaxy


Just like most transformer objects, there is an inverse_transform method that will get you back your original data. Here we must wrap row0 in a list to make it a 2D array.

In [91]:
title_ohe.inverse_transform([encoded_titles[0]])

array([['Guardians of the Galaxy']], dtype=object)

In [117]:
from_data = data['Title'].values
from_train = list(map(lambda x: x[0], title_ohe.inverse_transform(encoded_titles)))
np.unique(from_data == from_train)

array([ True])

In [154]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def find_text_columns(frame):
    values = [i for (i, t) in enumerate(frame.dtypes.values) if t.name == 'object']
    return frame.columns[values]

def find_numeric_columns(frame):
    values = [i for (i, t) in enumerate(frame.dtypes.values) if t.name != 'object']
    return frame.columns[values]
    

text_si_step = ('si', SimpleImputer(strategy='constant', fill_value='MISSING'))
text_ohe_step = ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
text_pipe = Pipeline([si_step, ohe_step])
text_cols = find_text_columns(data)

numeric_si_step = ('si', SimpleImputer(strategy='median'))
numeric_ss_step = ('ss', StandardScaler())
numeric_pipe = Pipeline([numeric_si_step, numeric_ss_step])
numeric_cols = find_number_columns(data)

transformers = [
    ('text', text_pipe, text_cols),
    ('num', numeric_pipe, numeric_cols)
]
ct = ColumnTransformer(transformers=transformers)

In [225]:
from sklearn.linear_model import Ridge
import inspect

def normalize(array):
    median = 0
    nans = []
    for (i, value) in enumerate(array):
        if np.isnan(value) or value == 0.0:
            nans.append(i)
        else:
            median += value
    
    median /= len(array)
    for i in nans:
        array[i] = median
        
    return array



revenue = normalize(revenue)
ml_pipe = Pipeline([('transform', ct), ('ridge', Ridge())])

In [226]:
for i in range(5):
    ml_pipe.fit(data, revenue)
    ml_pipe.score(data, revenue)

In [228]:
prediction = ml_pipe.predict(data)
# sorted(prediction)
diff = abs(prediction - revenue)
sorted([d for d in diff])
# percentage_difference = abs(prediction - revenue) / revenue) * 100
# percentage_difference
# avg_percentage_difference = sum(percentage_difference) / len(percentage_difference)
# avg_percentage_difference

[0.01277792168353642,
 0.023076816481769236,
 0.03376038172817175,
 0.04168484215073143,
 0.0677951149650653,
 0.07060049896578846,
 0.08609510930219244,
 0.11545679794421915,
 0.13773308185125188,
 0.1659485995775185,
 0.18248742145201646,
 0.18990324785715984,
 0.19071456465513847,
 0.19932136303870607,
 0.23239799205414613,
 0.2425639070005552,
 0.24576193348973163,
 0.25440928961982934,
 0.25909979468681144,
 0.2664284665856718,
 0.2785845088326191,
 0.3283532818152253,
 0.3318395851567004,
 0.40633002723041045,
 0.40834518136050946,
 0.41713600728755296,
 0.42624285756221525,
 0.4296840898093919,
 0.4608803297111024,
 0.46886285515858717,
 0.48923992120799653,
 0.48994035470536446,
 0.4917351854765144,
 0.4967019480856649,
 0.5088929427750344,
 0.5199116286849921,
 0.5356241021541415,
 0.5606488841451345,
 0.5672586391319356,
 0.5739050807006194,
 0.5849590707842367,
 0.5955155217201666,
 0.6048843901036776,
 0.6350370202861413,
 0.6542430600019662,
 0.6561088781602376,
 0.6807024